In [115]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.utils import shuffle

from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
#from sklearn.metrics import plot_roc_curve
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier

from sklearn.svm import SVC
from sklearn.svm import LinearSVC

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing

In [116]:
df = pd.read_csv('dataset.csv')
df = shuffle(df, random_state = 42)
df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
373,Acne,skin_rash,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4916,Acne,skin_rash,pus_filled_pimples,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1550,Hyperthyroidism,fatigue,mood_swings,weight_loss,restlessness,sweating,diarrhoea,fast_heart_rate,excessive_hunger,muscle_weakness,irritability,abnormal_menstruation,NaN,NaN,NaN,NaN,NaN,NaN
3081,AIDS,muscle_wasting,patches_in_throat,high_fever,extra_marital_contacts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3857,Chronic cholestasis,itching,vomiting,yellowish_skin,nausea,loss_of_appetite,abdominal_pain,yellowing_of_eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
df = pd.read_csv('dataset.csv')
severity = pd.read_csv('Symptom-severity.csv')
description = pd.read_csv('symptom_Description.csv')
precaution = pd.read_csv('symptom_precaution.csv')
updated_dataset = pd.read_csv('updated_dataset2.csv')

combined_df = pd.merge(df,description,on='Disease') #combining 2 dataframes

selected_columns = ['Disease', 'Age', 'Gender']
updated_dataset_filtered = updated_dataset[selected_columns]

combined_df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17,Description
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In humans, fungal infections occur when an inv..."
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In humans, fungal infections occur when an inv..."
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In humans, fungal infections occur when an inv..."
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In humans, fungal infections occur when an inv..."
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In humans, fungal infections occur when an inv..."


In [118]:
# Create a dictionary to map symptoms to their weights
symptom_mapping = pd.Series(severity['weight'].values, index=severity['Symptom']).to_dict()

# Example mapping
print(symptom_mapping)
#print(severity)

{'itching': 1, 'skin_rash': 3, 'nodal_skin_eruptions': 4, 'continuous_sneezing': 4, 'shivering': 5, 'chills': 3, 'joint_pain': 3, 'stomach_pain': 5, 'acidity': 3, 'ulcers_on_tongue': 4, 'muscle_wasting': 3, 'vomiting': 5, 'burning_micturition': 6, 'spotting_urination': 6, 'fatigue': 4, 'weight_gain': 3, 'anxiety': 4, 'cold_hands_and_feets': 5, 'mood_swings': 3, 'weight_loss': 3, 'restlessness': 5, 'lethargy': 2, 'patches_in_throat': 6, 'irregular_sugar_level': 5, 'cough': 4, 'high_fever': 7, 'sunken_eyes': 3, 'breathlessness': 4, 'sweating': 3, 'dehydration': 4, 'indigestion': 5, 'headache': 3, 'yellowish_skin': 3, 'dark_urine': 4, 'nausea': 5, 'loss_of_appetite': 4, 'pain_behind_the_eyes': 4, 'back_pain': 3, 'constipation': 4, 'abdominal_pain': 4, 'diarrhoea': 6, 'mild_fever': 5, 'yellow_urine': 4, 'yellowing_of_eyes': 4, 'acute_liver_failure': 6, 'fluid_overload': 4, 'swelling_of_stomach': 7, 'swelled_lymph_nodes': 6, 'malaise': 6, 'blurred_and_distorted_vision': 5, 'phlegm': 5, 'thr

In [119]:
import json
with open('symptomMap.json', 'r') as file:
    symptom_map = json.load(file)

In [120]:
combined_df = pd.merge(combined_df,precaution,on='Disease') #combining another one
combined_df.sample(5)

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,...,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17,Description,Precaution_1,Precaution_2,Precaution_3,Precaution_4
287,Hypothyroidism,fatigue,weight_gain,cold_hands_and_feets,mood_swings,dizziness,puffy_face_and_eyes,enlarged_thyroid,brittle_nails,swollen_extremeties,...,NaN,NaN,NaN,NaN,NaN,"Hypothyroidism, also called underactive thyroi...",reduce stress,exercise,eat healthy,get proper sleep
3208,Typhoid,chills,vomiting,fatigue,high_fever,headache,nausea,constipation,abdominal_pain,diarrhoea,...,NaN,NaN,NaN,NaN,NaN,An acute illness characterized by fever caused...,eat high calorie vegitables,antiboitic therapy,consult doctor,medication
1381,Common Cold,continuous_sneezing,chills,fatigue,cough,high_fever,headache,swelled_lymph_nodes,malaise,phlegm,...,runny_nose,congestion,chest_pain,loss_of_smell,muscle_pain,The common cold is a viral infection of your n...,drink vitamin c rich drinks,take vapour,avoid cold food,keep fever in check
909,Chicken pox,itching,skin_rash,fatigue,lethargy,high_fever,headache,loss_of_appetite,mild_fever,malaise,...,NaN,NaN,NaN,NaN,NaN,Chickenpox is a highly contagious disease caus...,use neem in bathing,consume neem leaves,take vaccine,avoid public places
3167,Malaria,chills,vomiting,high_fever,sweating,headache,nausea,diarrhoea,muscle_pain,NaN,...,NaN,NaN,NaN,NaN,NaN,An infectious disease caused by protozoan para...,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out


In [121]:
combined_df.isna().sum()

Disease            0
Symptom_1          0
Symptom_2          0
Symptom_3          0
Symptom_4        348
Symptom_5       1128
Symptom_6       1746
Symptom_7       2412
Symptom_8       2736
Symptom_9       2988
Symptom_10      3120
Symptom_11      3366
Symptom_12      3816
Symptom_13      4056
Symptom_14      4254
Symptom_15      4320
Symptom_16      4368
Symptom_17      4488
Description        0
Precaution_1       0
Precaution_2       0
Precaution_3     120
Precaution_4     120
dtype: int64

In [122]:
del combined_df['Symptom_4']    #deleting the columns which we dont need
del combined_df['Symptom_5']   
del combined_df['Symptom_6']   
del combined_df['Symptom_7']   
del combined_df['Symptom_8']   
del combined_df['Symptom_9']   
del combined_df['Symptom_10']   
del combined_df['Symptom_11']   
del combined_df['Symptom_12']   
del combined_df['Symptom_13']   
del combined_df['Symptom_14']   
del combined_df['Symptom_15']   
del combined_df['Symptom_16']
del combined_df['Symptom_17']   

In [123]:
combined_df = combined_df.fillna(method='ffill')
combined_df.columns

C:\Users\Samia Kazi\AppData\Local\Temp\ipykernel_9688\724263746.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df = combined_df.fillna(method='ffill')


Index(['Disease', 'Symptom_1', 'Symptom_2', 'Symptom_3', 'Description',
       'Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4'],
      dtype='object')

In [124]:
print(combined_df['Disease'].value_counts())

Disease
Fungal infection                           120
Hypothyroidism                             120
Hepatitis E                                120
Alcoholic hepatitis                        120
Tuberculosis                               120
Common Cold                                120
Pneumonia                                  120
Heart attack                               120
Varicose veins                             120
Hyperthyroidism                            120
Allergy                                    120
Hypoglycemia                               120
Osteoarthristis                            120
Arthritis                                  120
(vertigo) Paroymsal  Positional Vertigo    120
Acne                                       120
Urinary tract infection                    120
Psoriasis                                  120
Hepatitis D                                120
Hepatitis C                                120
Hepatitis B                                120
hepat

In [125]:
combined_df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Description,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,"In humans, fungal infections occur when an inv...",bath twice,use detol or neem in bathing water,keep infected area dry,use clean cloths
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,"In humans, fungal infections occur when an inv...",bath twice,use detol or neem in bathing water,keep infected area dry,use clean cloths
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,"In humans, fungal infections occur when an inv...",bath twice,use detol or neem in bathing water,keep infected area dry,use clean cloths
3,Fungal infection,itching,skin_rash,dischromic _patches,"In humans, fungal infections occur when an inv...",bath twice,use detol or neem in bathing water,keep infected area dry,use clean cloths
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,"In humans, fungal infections occur when an inv...",bath twice,use detol or neem in bathing water,keep infected area dry,use clean cloths


In [126]:
for col in combined_df.columns:
    if combined_df[col].dtype == 'object':  # Check if the column is of string type
        combined_df[col] = combined_df[col].str.strip()
combined_df.head()
# Save the updated dataset
output_file_path = 'C:\\health-awareness-portal\\public\\Server\\models\\combined_df.csv'  # Replace with your desired output path
combined_df.to_csv(output_file_path, index=False)

print(f"Dataset updated with Severity and Duration columns and saved to {output_file_path}")

Dataset updated with Severity and Duration columns and saved to C:\health-awareness-portal\public\Server\models\combined_df.csv


In [127]:
print(combined_df)
combined_df.isna().sum()

                                      Disease            Symptom_1  \
0                            Fungal infection              itching   
1                            Fungal infection            skin_rash   
2                            Fungal infection              itching   
3                            Fungal infection              itching   
4                            Fungal infection              itching   
...                                       ...                  ...   
4555  (vertigo) Paroymsal  Positional Vertigo             vomiting   
4556                                     Acne            skin_rash   
4557                  Urinary tract infection  burning_micturition   
4558                                Psoriasis            skin_rash   
4559                                 Impetigo            skin_rash   

                 Symptom_2             Symptom_3  \
0                skin_rash  nodal_skin_eruptions   
1     nodal_skin_eruptions   dischromic _patches   
2  

Disease         0
Symptom_1       0
Symptom_2       0
Symptom_3       0
Description     0
Precaution_1    0
Precaution_2    0
Precaution_3    0
Precaution_4    0
dtype: int64

In [128]:
print(combined_df[['Symptom_1', 'Symptom_2', 'Symptom_3']].stack().unique())  # Unique symptoms in your dataset
print(symptom_map.keys())  # Keys in your symptomMap.json

['itching' 'skin_rash' 'nodal_skin_eruptions' 'dischromic _patches'
 'continuous_sneezing' 'shivering' 'chills' 'watering_from_eyes'
 'stomach_pain' 'acidity' 'ulcers_on_tongue' 'vomiting' 'yellowish_skin'
 'nausea' 'burning_micturition' 'loss_of_appetite' 'abdominal_pain'
 'indigestion' 'muscle_wasting' 'patches_in_throat' 'high_fever'
 'extra_marital_contacts' 'sunken_eyes' 'dehydration' 'diarrhoea'
 'fatigue' 'cough' 'breathlessness' 'headache'
 'blurred_and_distorted_vision' 'back_pain' 'weakness_in_limbs'
 'neck_pain' 'dizziness' 'weakness_of_one_body_side' 'altered_sensorium'
 'weight_loss' 'sweating' 'lethargy' 'joint_pain' 'dark_urine'
 'swelling_of_stomach' 'chest_pain' 'cramps' 'bruising' 'obesity'
 'weight_gain' 'cold_hands_and_feets' 'mood_swings' 'restlessness'
 'anxiety' 'knee_pain' 'hip_joint_pain' 'muscle_weakness' 'stiff_neck'
 'swelling_joints' 'movement_stiffness' 'spinning_movements'
 'pus_filled_pimples' 'blackheads' 'scurring' 'bladder_discomfort'
 'foul_smell_of 

In [129]:
# Remove leading/trailing spaces and replace underscores with spaces (if needed)
# Clean the dataset by ensuring that strip and replace only apply to string values
combined_df[['Symptom_1', 'Symptom_2', 'Symptom_3']] = combined_df[['Symptom_1', 'Symptom_2', 'Symptom_3']].applymap(
    lambda x: x.strip().replace(' ', '_') if isinstance(x, str) else x
)
for col in combined_df.columns:
    if combined_df[col].dtype == 'object':  # Check if the column is of string type
        combined_df[col] = combined_df[col].str.strip()
combined_df.head()
# Save the updated dataset
output_file_path = 'C:\\health-awareness-portal\\public\\Server\\models\\combined_df1.csv'  # Replace with your desired output path
combined_df.to_csv(output_file_path, index=False)

print(f"Dataset updated with Severity and Duration columns and saved to {output_file_path}")

# Clean dataset symptoms
#symptom_map_clean = {key.strip().replace(' ', '_'): value for key, value in symptom_map.items()}

'''combined_df[['Symptom_1', 'Symptom_2', 'Symptom_3']] = combined_df[['Symptom_1', 'Symptom_2', 'Symptom_3']].apply(
    lambda x: x.str.strip().str.replace(r'\s+', '_', regex=True).str.lower()
)
# Normalize mapping keys
symptom_mapp = {k.lower().strip(): v for k, v in symptom_map.items()}'''

C:\Users\Samia Kazi\AppData\Local\Temp\ipykernel_9688\836551224.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  combined_df[['Symptom_1', 'Symptom_2', 'Symptom_3']] = combined_df[['Symptom_1', 'Symptom_2', 'Symptom_3']].applymap(


Dataset updated with Severity and Duration columns and saved to C:\health-awareness-portal\public\Server\models\combined_df1.csv


"combined_df[['Symptom_1', 'Symptom_2', 'Symptom_3']] = combined_df[['Symptom_1', 'Symptom_2', 'Symptom_3']].apply(\n    lambda x: x.str.strip().str.replace(r'\\s+', '_', regex=True).str.lower()\n)\n# Normalize mapping keys\nsymptom_mapp = {k.lower().strip(): v for k, v in symptom_map.items()}"

In [ ]:
combined_df1 = pd.read_csv('combined_df1.csv')
combined_df1

,Disease,Symptom_1,Symptom_2,Symptom_3,Description,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,"In humans, fungal infections occur when an inv...",bath twice,use detol or neem in bathing water,keep infected area dry,use clean cloths
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic__patches,"In humans, fungal infections occur when an inv...",bath twice,use detol or neem in bathing water,keep infected area dry,use clean cloths
2,Fungal infection,itching,nodal_skin_eruptions,dischromic__patches,"In humans, fungal infections occur when an inv...",bath twice,use detol or neem in bathing water,keep infected area dry,use clean cloths
3,Fungal infection,itching,skin_rash,dischromic__patches,"In humans, fungal infections occur when an inv...",bath twice,use detol or neem in bathing water,keep infected area dry,use clean cloths
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,"In humans, fungal infections occur when an inv...",bath twice,use detol or neem in bathing water,keep infected area dry,use clean cloths
...,...,...,...,...,...,...,...,...,...
4555,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,Benign paroxysmal positional vertigo (BPPV) is...,lie down,avoid sudden change in body,avoid abrupt head movment,relax
4556,Acne,skin_rash,pus_filled_pimples,blackheads,"Acne vulgaris is the formation of comedones, p...",bath twice,avoid fatty spicy food,drink plenty of water,avoid too many products
4557,Urinary tract infection,burning_micturition,bladder_discomfort,foul_smell_of_urine,Urinary tract infection: An infection of the k...,drink plenty of water,increase vitamin c intake,drink cranberry juice,take probiotics
4558,Psoriasis,skin_rash,joint_pain,skin_peeling,Psoriasis is a common skin disorder that forms...,wash hands with warm soapy water,stop bleeding using pressure,consult doctor,salt baths


In [131]:
print(combined_df1[['Symptom_1', 'Symptom_2', 'Symptom_3']])

                Symptom_1             Symptom_2             Symptom_3
0                 itching             skin_rash  nodal_skin_eruptions
1               skin_rash  nodal_skin_eruptions   dischromic__patches
2                 itching  nodal_skin_eruptions   dischromic__patches
3                 itching             skin_rash   dischromic__patches
4                 itching             skin_rash  nodal_skin_eruptions
...                   ...                   ...                   ...
4555             vomiting              headache                nausea
4556            skin_rash    pus_filled_pimples            blackheads
4557  burning_micturition    bladder_discomfort   foul_smell_of_urine
4558            skin_rash            joint_pain          skin_peeling
4559            skin_rash            high_fever               blister

[4560 rows x 3 columns]


In [132]:
# Map symptoms to numerical values using symptom_mapping.json
combined_df1['Symptom_1'] = combined_df1['Symptom_1'].map(symptom_map)
combined_df1['Symptom_2'] = combined_df1['Symptom_2'].map(symptom_map)
combined_df1['Symptom_3'] = combined_df1['Symptom_3'].map(symptom_map)
print(combined_df1[['Symptom_1', 'Symptom_2', 'Symptom_3']])

      Symptom_1  Symptom_2  Symptom_3
0             1          2          3
1             2          3        103
2             1          3        103
3             1          2        103
4             1          2          3
...         ...        ...        ...
4555         12         32         35
4556          2        122        123
4557         13         90         91
4558          2          7        125
4559          2         26        129

[4560 rows x 3 columns]


In [133]:
print(combined_df1[['Symptom_1', 'Symptom_2', 'Symptom_3']].stack().unique())

[  1   2   3 103   4   5   6 104   8   9  10  12  33  35  13  36  40  31
  11  23  26  76  27  30  41  15  25  28  32  50  38  58  64  65  88  99
  20  29  22   7  34  47  57  66  67  68  16  18  19  21  17  79  80  81
  82  83  84  85 122 123 124  90  91  92 125 126 129 130]


In [134]:
combined_df1.isna().sum()

Disease         0
Symptom_1       0
Symptom_2       0
Symptom_3       0
Description     0
Precaution_1    0
Precaution_2    0
Precaution_3    0
Precaution_4    0
dtype: int64

In [135]:
# Separate features and target
X = combined_df1[['Symptom_1', 'Symptom_2', 'Symptom_3']].astype(int)  # Ensure integer type
y = combined_df1['Disease']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier()
#model = RandomForestClassifier(random_state=42, n_estimators=100)
model.fit(X_train, y_train)
# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Model Accuracy: 95.07%
Classification Report:
                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00        21
                                   AIDS       1.00      1.00      1.00        28
                                   Acne       1.00      1.00      1.00        32
                    Alcoholic hepatitis       1.00      1.00      1.00        28
                                Allergy       1.00      1.00      1.00        28
                              Arthritis       1.00      1.00      1.00        23
                       Bronchial Asthma       0.91      1.00      0.95        29
                   Cervical spondylosis       1.00      1.00      1.00        21
                            Chicken pox       1.00      1.00      1.00        11
                    Chronic cholestasis       1.00      1.00      1.00        27
                            Common Cold       1.00      1.00  

In [136]:
combined_df1.isna().sum()

Disease         0
Symptom_1       0
Symptom_2       0
Symptom_3       0
Description     0
Precaution_1    0
Precaution_2    0
Precaution_3    0
Precaution_4    0
dtype: int64

In [137]:
def performance_evaluator(model, X_test, y_test):
    """
    model: Load the trained model
    X_test: test data
    y_test: Actual value
    
    """
    
    y_predicted = model.predict(X_test)
    
    precision = precision_score(y_test, y_predicted,average='micro')*100
    
    accuracy = accuracy_score(y_test, y_predicted)*100
    
    f1 = f1_score(y_test, y_predicted, average='macro')*100
    
    recall = recall_score(y_test, y_predicted, average='macro')*100

    print('precision----->', precision) 
    print('\n************************')
    print('Accuracy----->', accuracy)
    print('\n************************')
    print('F1 Score----->', f1)
    print('\n************************')
    print('Recall----->', recall)
    print('\n************************')
    return accuracy, precision, f1, recall

In [138]:
combined_df1.isna().sum()

Disease         0
Symptom_1       0
Symptom_2       0
Symptom_3       0
Description     0
Precaution_1    0
Precaution_2    0
Precaution_3    0
Precaution_4    0
dtype: int64

In [139]:
SVM_init = SVC()
model_SVM_init = SVM_init.fit(X_train, y_train)
_1, _2, _3, _4 = performance_evaluator(model_SVM_init, X_test, y_test)

precision-----> 67.54385964912281

************************
Accuracy-----> 67.54385964912281

************************
F1 Score-----> 64.0841208412635

************************
Recall-----> 69.0015245834603

************************


In [140]:
GBC_model = GradientBoostingClassifier()
GBC_model.fit(X_train, y_train)
_1, _2, _3, _4 = performance_evaluator(GBC_model, X_test, y_test)

precision-----> 94.95614035087719

************************
Accuracy-----> 94.95614035087719

************************
F1 Score-----> 92.8662664849619

************************
Recall-----> 94.32051997000173

************************


In [141]:
import joblib
# Save the trained model
joblib.dump(model, 'Symptom_Based_Disease_Prediction.pkl')
print("Model saved as 'Symptom_Based_Disease_Prediction.pkl'")

Model saved as 'Symptom_Based_Disease_Prediction.pkl'
